<a target="_blank" href="https://colab.research.google.com/github/IngCarlaPezzone/tensorflow-1-public/blob/main/C1/W4/ungraded_labs/C1_W4_Lab_2_image_generator_with_validation_traducido.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Ungraded Lab: ImageDataGenerator con un conjunto de validación

En este laboratorio, seguirá utilizando la clase `ImageDataGenerator` para preparar el conjunto de datos `Horses or Humans`. Esta vez, agregará un conjunto de validación para que también pueda medir el desempeño del modelo en datos que no ha visto.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

# Ungraded Lab: ImageDataGenerator with a Validation Set

In this lab, you will continue using the `ImageDataGenerator` class to prepare the `Horses or Humans` dataset. This time, you will add a validation set so you can also measure how well the model performs on data it hasn't seen.

**NOTA IMPORTANTE:** Este cuaderno está diseñado para ejecutarse como Colab. Ejecutarlo en su máquina local puede dar lugar a que algunos de los bloques de código arrojen errores.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

**IMPORTANT NOTE:** This notebook is designed to run as a Colab. Running it on your local machine might result in some of the code blocks throwing errors.

In [ ]:
# Instale este paquete para utilizar la GPU de Colab para la formación
!apt install --allow-change-held-packages libcudnn8=8.4.1.50-1+cuda11.6

Ejecute los siguientes bloques de código para descargar los conjuntos de datos `caballo-o-humano.zip` y `validación-caballo-o-humano.zip` respectivamente.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

Run the code blocks below to download the datasets `horse-or-human.zip` and `validation-horse-or-human.zip` respectively.

In [ ]:
# Descargar el conjunto de entrenamiento
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

In [ ]:
# Descargar el conjunto de validación
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip

Luego descomprime ambos archivos.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

Then unzip both archives.

In [ ]:
import zipfile

# Descomprimir el conjunto de entrenamiento
local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')

# Descomprimir conjunto de validación
local_zip = './validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./validation-horse-or-human')

zip_ref.close()

Al igual que en el laboratorio anterior, definirá los directorios que contienen sus imágenes. Esta vez, incluirá los que tienen datos de validación.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

Similar to the previous lab, you will define the directories containing your images. This time, you will include those with validation data.

In [ ]:
import os

# Directorio con fotos de caballos de entrenamiento
train_horse_dir = os.path.join('./horse-or-human/horses')

# Directorio con fotos de humanos en entrenamiento
train_human_dir = os.path.join('./horse-or-human/humans')

# Directorio con imágenes de caballos de validación
validation_horse_dir = os.path.join('./validation-horse-or-human/horses')

# Directorio con fotos de humanos de validación
validation_human_dir = os.path.join('./validation-horse-or-human/humans')

Ahora vea cómo son los nombres de los archivos en estos directorios:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

Now see what the filenames look like in these directories:

In [ ]:
train_horse_names = os.listdir(train_horse_dir)
print(f'TRAIN SET HORSES: {train_horse_names[:10]}')

train_human_names = os.listdir(train_human_dir)
print(f'TRAIN SET HUMANS: {train_human_names[:10]}')

validation_horse_names = os.listdir(validation_horse_dir)
print(f'VAL SET HORSES: {validation_horse_names[:10]}')

validation_human_names = os.listdir(validation_human_dir)
print(f'VAL SET HUMANS: {validation_human_names[:10]}')

Puede averiguar el número total de imágenes de caballos y humanos en los directorios:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

You can find out the total number of horse and human images in the directories:

In [ ]:
print(f'total training horse images: {len(os.listdir(train_horse_dir))}')
print(f'total training human images: {len(os.listdir(train_human_dir))}')
print(f'total validation horse images: {len(os.listdir(validation_horse_dir))}')
print(f'total validation human images: {len(os.listdir(validation_human_dir))}')

Ahora echa un vistazo a algunas imágenes para tener una mejor idea de su aspecto. Primero, configura los parámetros de `matplotlib`:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

Now take a look at a few pictures to get a better sense of what they look like. First, configure the `matplotlib` parameters:

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Parámetros para nuestro gráfico; vamos a dar salida a las imágenes en una configuración 4x4
nrows = 4
ncols = 4

# Índice para iterar sobre las imágenes
pic_index = 0

Ahora, muestre un lote de 8 imágenes de caballos y 8 de humanos. Puede volver a ejecutar la celda para ver un nuevo lote cada vez:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

Now, display a batch of 8 horse and 8 human pictures. You can rerun the cell to see a fresh batch each time:

In [ ]:
# Configurar matplotlib fig, y dimensionarlo para que quepa en 4x4 fotos
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_horse_pix = [os.path.join(train_horse_dir, fname) 
                for fname in train_horse_names[pic_index-8:pic_index]]
next_human_pix = [os.path.join(train_human_dir, fname) 
                for fname in train_human_names[pic_index-8:pic_index]]

for i, img_path in enumerate(next_horse_pix+next_human_pix):
    # Configurar el subplot; los índices del subplot comienzan en 1
    sp = plt.subplot(nrows, ncols, i + 1)
    sp.axis('Off') # No mostrar los ejes (ni las líneas de la cuadrícula)

    img = mpimg.imread(img_path)
    plt.imshow(img)

plt.show()


## Construir un pequeño modelo desde cero

Definirás la misma arquitectura del modelo que antes:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

## Building a Small Model from Scratch

You will define the same model architecture as before:

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Tenga en cuenta la forma de entrada es el tamaño deseado de la imagen 300x300 con 3 bytes de color
    # Esta es la primera convolución
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # La segunda convolución
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # La tercer convolución
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # La cuarta convolución
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # La quinta convolución
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Aplanar los resultados para alimentar una DNN
    tf.keras.layers.Flatten(),
    # Capa oculta de 512 neuronas
    tf.keras.layers.Dense(512, activation='relu'),
    # Sólo 1 neurona de salida. Contendrá un valor de 0-1 
    # donde 0 para una clase ('caballos') y 1 para la otra ('humanos')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

Puede revisar la arquitectura de la red y las formas de salida con `model.summary()`.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

You can review the network architecture and the output shapes with `model.summary()`.

In [ ]:
model.summary()

También utilizará la misma configuración de compilación que antes:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

You will also use the same compile settings as before:

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

### Preprocesamiento de datos

Ahora configurará los generadores de datos. En su mayor parte será lo mismo que la última vez, pero observe el código adicional para preparar también los datos de validación. Tendrá que ser instanciado por separado y también escalado para tener `[0,1]` rango de valores de píxeles.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

### Data Preprocessing

Now you will setup the data generators. It will mostly be the same as last time but notice the additional code to also prepare the validation data. It will need to be instantiated separately and also scaled to have `[0,1]` range of pixel values.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Todas las imágenes serán reescaladas por 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Flujo de imágenes de entrenamiento en lotes de 128 utilizando el generador train_datagen
train_generator = train_datagen.flow_from_directory(
        './horse-or-human/',  # Este es el directorio de origen para las imágenes de entrenamiento
        target_size=(300, 300),  # Todas las imágenes serán redimensionadas a 300x300
        batch_size=128,
        # Como se utiliza la pérdida binaria_crossentropy, se necesitan etiquetas binarias
        class_mode='binary')

# Flujo de imágenes de validación en lotes de 128 utilizando el generador validation_datagen
validation_generator = validation_datagen.flow_from_directory(
        './validation-horse-or-human/',  # Este es el directorio de origen para las imágenes de validación
        target_size=(300, 300),  # Todas las imágenes se redimensionarán a 300x300
        batch_size=32,
        # Como se utiliza la pérdida binaria_crossentropy, se necesitan etiquetas binarias
        class_mode='binary')

### Entrenamiento
Ahora entrena el modelo durante 15 épocas. Aquí, pasarás parámetros para `validation_data` y `validation_steps`. Con estos, usted notará salidas adicionales en las declaraciones de impresión: `val_loss` y `val_accuracy`. Observe que a medida que entrena con más épocas, su precisión de entrenamiento puede aumentar pero su precisión de validación disminuye. Esto puede ser un signo de sobreajuste y necesitas evitar que tu modelo llegue a este punto.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

### Training
Now train the model for 15 epochs. Here, you will pass parameters for `validation_data` and `validation_steps`. With these, you will notice additional outputs in the print statements: `val_loss` and `val_accuracy`. Notice that as you train with more epochs, your training accuracy might go up but your validation accuracy goes down. This can be a sign of overfitting and you need to prevent your model from reaching this point.

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)

### Predicción del modelo

Ahora eche un vistazo a la ejecución de una predicción utilizando el modelo. Este código le permitirá elegir 1 o más archivos de su sistema de archivos, cargarlos y ejecutarlos a través del modelo, dando una indicación de si el objeto es un caballo o un humano.

**Nota importante:** Debido a algunos problemas de compatibilidad, el siguiente bloque de código dará lugar a un error después de seleccionar las imágenes a cargar si está ejecutando este cuaderno como un `Colab` en el navegador `Safari`. Para todos los demás navegadores, continúe con el siguiente bloque de código e ignore el que le sigue.

Para los usuarios de Safari: por favor, comente u omita el bloque de código de abajo, descomente el siguiente bloque de código y ejecútelo.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

### Model Prediction

Now take a look at actually running a prediction using the model. This code will allow you to choose 1 or more files from your file system, upload them, and run them through the model, giving an indication of whether the object is a horse or a human.

**Important Note:** Due to some compatibility issues, the following code block will result in an error after you select the images(s) to upload if you are running this notebook as a `Colab` on the `Safari` browser. For all other browsers, continue with the next code block and ignore the next one after it.

_For Safari users: please comment out or skip the code block below, uncomment the next code block and run it._

In [ ]:
## BLOQUE DE CÓDIGO PARA NAVEGADORES QUE NO SEAN SAFARI
## USUARIOS DE SAFARI: POR FAVOR, OMITA ESTE BLOQUE Y EJECUTE EL SIGUIENTE EN SU LUGAR

import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
    # predicting images
    path = '/content/' + fn
    img = image.load_img(path, target_size=(300, 300))
    x = image.img_to_array(img)
    x /= 255
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    print(classes[0])
    if classes[0]>0.5:
    print(fn + " is a human")
    else:
    print(fn + " is a horse")
 

Los usuarios de `Safari` tendrán que cargar las imágenes manualmente en su espacio de trabajo. Por favor, siga las instrucciones, descomente el bloque de código de abajo y ejecútelo.

Instrucciones para subir imágenes manualmente en un Colab:

1. Seleccione el icono de la `carpeta` en la `barra de menú` de la izquierda.
2. Haga clic en la `carpeta con una flecha apuntando hacia arriba` llamada `..`.
3. Haga clic en la `carpeta` llamada `tmp`.
4. Dentro de la carpeta `tmp`, `crea una nueva carpeta` llamada `imágenes`. Verás la opción `Nueva carpeta` haciendo clic en el botón de menú de los `3 puntos verticales` junto a la carpeta `tmp`.
5. Dentro de la nueva carpeta `imágenes`, sube una(s) imagen(es) de tu elección, preferiblemente de un caballo o de un humano. Arrastra y suelta la(s) imagen(es) encima de la carpeta `images`.
6. Descomente y ejecute el bloque de código que aparece a continuación. 

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

`Safari` users will need to upload the images(s) manually in their workspace. Please follow the instructions, uncomment the code block below and run it.

Instructions on how to upload image(s) manually in a Colab:

1. Select the `folder` icon on the left `menu bar`.
2. Click on the `folder with an arrow pointing upwards` named `..`
3. Click on the `folder` named `tmp`.
4. Inside of the `tmp` folder, `create a new folder` called `images`. You'll see the `New folder` option by clicking the `3 vertical dots` menu button next to the `tmp` folder.
5. Inside of the new `images` folder, upload an image(s) of your choice, preferably of either a horse or a human. Drag and drop the images(s) on top of the `images` folder.
6. Uncomment and run the code block below. 

In [ ]:
# # CODE BLOCK FOR SAFARI USERS

# import numpy as np
# from keras.preprocessing import image
# import os

# images = os.listdir("/tmp/images")

# print(images)

# for i in images:
#  print()
#  # predicting images
#  path = '/tmp/images/' + i
#  img = image.load_img(path, target_size=(300, 300))
#  x = image.img_to_array(img)
#  x /= 255
#  x = np.expand_dims(x, axis=0)

#  images = np.vstack([x])
#  classes = model.predict(images, batch_size=10)
#  print(classes[0])
#  if classes[0]>0.5:
#    print(i + " is a human")
#  else:
#    print(i + " is a horse")

### Visualización de representaciones intermedias

Al igual que antes, se puede trazar cómo se transforman las características a medida que pasa por cada capa.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

### Visualizing Intermediate Representations

As before, you can plot how the features are transformed as it goes through each layer.

In [ ]:
import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Definir un nuevo modelo que tomará una imagen como entrada, y emitirá
# representaciones intermedias para todas las capas del modelo anterior después de
# la primera.
successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

# Prepara una imagen de entrada aleatoria del conjunto de entrenamiento.
horse_img_files = [os.path.join(train_horse_dir, f) for f in train_horse_names]
human_img_files = [os.path.join(train_human_dir, f) for f in train_human_names]
img_path = random.choice(horse_img_files + human_img_files)

img = load_img(img_path, target_size=(300, 300))  # esta es una imagen PIL
x = img_to_array(img)  # matriz Numpy con forma (300, 300, 3)
x = x.reshape((1,) + x.shape)  # Matriz Numpy con forma (1, 300, 300, 3)

# Escala por  1/255
x /= 255

# Pasar la imagen por la red, obteniendo así todas
# representaciones intermedias de esta imagen.
successive_feature_maps = visualization_model.predict(x)

# Estos son los nombres de las capas, por lo que puede tenerlos como parte de la trama
layer_names = [layer.name for layer in model.layers[1:]]

# Visualizar las representaciones
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
    if len(feature_map.shape) == 4:

    # Sólo se hace esto para las capas conv / maxpool, no para las capas totalmente conectadas
    n_features = feature_map.shape[-1]  # number of features in feature map

    # El mapa de características tiene forma (1, size, size, n_features)
    size = feature_map.shape[1]

    # Mosaico de las imágenes en esta matriz
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
        x = feature_map[0, :, :, i]
        x -= x.mean()
        x /= x.std()
        x *= 64
        x += 128
        x = np.clip(x, 0, 255).astype('uint8')

        # Colocar cada filtro en esta gran cuadrícula horizontal
        display_grid[:, i * size : (i + 1) * size] = x
    
    # Mostrar la grilla
    scale = 20. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

## Limpiar

Antes de ejecutar el siguiente ejercicio, ejecute la siguiente celda para terminar el kernel y liberar recursos de memoria:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

## Clean Up

Before running the next exercise, run the following cell to terminate the kernel and free memory resources:

In [ ]:
# import os, signal
# os.kill(os.getpid(), signal.SIGKILL)